The purpose of this script is to find fads or "hot topics" in fMRI research over time. It uses Kleinberg's "bursty stream analysis" to identify topics that are associated with a burst of popularity and the time courses of their popularity.

Kleinberg, 2002. Bursty and Hierarchical Structure in Streams
https://www.cs.cornell.edu/home/kleinber/bhs.pdf



In [1]:
from Bio import Entrez

#search PubMed for all articles with "fMRI" in the title or abstract
Entrez.email = 'marinsek@dyns.ucsb.edu'
handle = Entrez.esearch(db='pubmed', 
                        sort='recent',
                        term='fMRI')
record = Entrez.read(handle)
#record is a list of article IDs that matched the query
record["IdList"]
